#### https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/images/transfer_learning_with_hub.ipynb#scrollTo=3n0Wb9ylKd8R


https://tfhub.dev/google/inaturalist/inception_v3/feature_vector/4

### Plant Disease Detection Tutorial



In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
import sys
import tensorflow_hub as hub
import os
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import time
sys.path.insert(0,'F:/Nilgai_photo_database/Nilgai Classifier/tf/code')
sys.path.insert(0,'F:/Nilgai_photo_database/Nilgai Classifier/tf/code/binary_class')
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers
from mycallback import CollectBatchStats
from fine_incptv3 import train_inv3
from tensorflow.keras.applications.inception_v3 import InceptionV3
from binary_datagen import binary_datagen
from binary_predict import bi_predict
from binary_evaluate import eval_test

import re
tf.keras.backend.clear_session()

# verify TensorFlow version
print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.test.is_gpu_available() else "NOT AVAILABLE")

Version:  2.2.0
Eager mode:  True
Hub version:  0.9.0.dev
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
GPU is available


In [2]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

# configuration settings
MODEL_NAME = 'Bi_Nilgai'
BATCH_SIZE = 32  #128
EPOCHS_1 = 5
IMAGE_SIZE = (299,299)
LEARNING_RATE = 0.001

# Train sets
# TRAIN_CSV = r'F:\Nilgai_photo_database\Nilgai Classifier\tf\data\binary_nil\100K_train_balanced_nil_data.csv'
TRAIN_CSV = r'F:\Nilgai_photo_database\Nilgai Classifier\tf\data\binary_nil\1K_train_balanced_nil_data.csv'

# Val sets
VAL_CSV = r'F:\Nilgai_photo_database\Nilgai Classifier\tf\data\binary_nil\10K_val_balanced_nil_data.csv'
# VAL_CSV = r'F:\Nilgai_photo_database\Nilgai Classifier\tf\data\binary_nil\1K_val_balanced_nil_data.csv'

# Test Sets
# TEST_CSV = r'F:\Nilgai_photo_database\Nilgai Classifier\tf\data\binary\5K_val_balanced_binary_data.csv'
TEST_CSV = r'F:\Nilgai_photo_database\Nilgai Classifier\tf\data\binary_nil\10K_test_weighted_nil_data.csv'

MODULE_HANDLE = "https://tfhub.dev/google/inaturalist/inception_v3/feature_vector/4"

# call training and validation generators
train_generator, validation_generator = binary_datagen(TRAIN_CSV, VAL_CSV, BATCH_SIZE, IMAGE_SIZE)

Num GPUs Available:  1
Found 1000 validated image filenames belonging to 2 classes.
Found 10000 validated image filenames belonging to 2 classes.


In [3]:
timestr = datetime.now().strftime("%m%d%Y-%H%M%S")

os.chdir('F:/Nilgai_photo_database/Nilgai Classifier/tf/saved_model')
new_dir = MODEL_NAME + timestr

# Create target Directory if don't exist
if not os.path.exists(new_dir):
    os.mkdir(new_dir)
    print("Directory ", new_dir, " Created ")
else:    
    print("Directory ", new_dir, " already exists")

checkpoint_path = new_dir + "/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)


Directory  Bi_Nilgai05202020-173418  Created 


In [4]:

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, monitor='val_acc', save_best_only=True, save_weights_only=True, mode='max',
                                                 verbose=1)

cvs_name = MODEL_NAME + '_bi_nil_model_log.csv'                                                 
cvs_logger = tf.keras.callbacks.CSVLogger(cvs_name, separator=',', append=True)

batch_stats = CollectBatchStats()

# Set steps per epoch
steps_per_epoch = np.ceil(train_generator.samples/train_generator.batch_size)
val_steps = np.ceil(validation_generator.samples/validation_generator.batch_size)

# First stage: transfer learning using HUB (iNat)

In [5]:
feature_extractor_layer = hub.KerasLayer(MODULE_HANDLE,
        input_shape=(299,299,3))

feature_extractor_layer.trainable = False

    # setup model
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(299,299,3)),
    feature_extractor_layer,
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(1, activation='sigmoid')
    ])

model.build((None,)+(299,299)+(3,))

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=['acc'])

model.save_weights(checkpoint_path.format(epoch=0))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 2048)              21802784  
_________________________________________________________________
dropout (Dropout)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 1)                 2049      
Total params: 21,804,833
Trainable params: 2,049
Non-trainable params: 21,802,784
_________________________________________________________________


In [6]:
history_tf = model.fit(
            train_generator,
            epochs=EPOCHS_1,
            validation_data=validation_generator,
            steps_per_epoch=steps_per_epoch,
            validation_steps=val_steps,
            verbose=1,
            workers=4, 
            callbacks=[cp_callback, batch_stats, cvs_logger]
            )

Epoch 1/5
3125/3125 [==============================] - ETA: 0s - loss: 0.5955 - acc: 0.8750
Epoch 00001: val_acc improved from -inf to 0.80470, saving model to Bi_Nilgai05202020-173418/cp-0001.ckpt
3125/3125 [==============================] - 2041s 653ms/step - loss: 0.5955 - acc: 0.8750 - val_loss: 0.5932 - val_acc: 0.8047
Epoch 2/5
3125/3125 [==============================] - ETA: 0s - loss: 0.6515 - acc: 0.6875
Epoch 00002: val_acc improved from 0.80470 to 0.81370, saving model to Bi_Nilgai05202020-173418/cp-0002.ckpt
3125/3125 [==============================] - 1782s 570ms/step - loss: 0.6515 - acc: 0.6875 - val_loss: 0.5865 - val_acc: 0.8137
Epoch 3/5
3125/3125 [==============================] - ETA: 0s - loss: 0.6611 - acc: 0.7188
Epoch 00003: val_acc improved from 0.81370 to 0.81810, saving model to Bi_Nilgai05202020-173418/cp-0003.ckpt
3125/3125 [==============================] - 1801s 576ms/step - loss: 0.6611 - acc: 0.7188 - val_loss: 0.5839 - val_acc: 0.8181
Epoch 4/5
3125/3

# Second stage fine-tunning

In [7]:
EPOCHS_2 = EPOCHS_1 + 5
LEARNING_RATE = 0.0001

# load last checkpoint
# model = tf.keras.models.load_model(model_path, compile=True)

latest = tf.train.latest_checkpoint(checkpoint_dir)
model.load_weights(latest)
latest

'Bi_Nilgai05202020-173418\\cp-0004.ckpt'

In [8]:
# from previsous model training event
feature_extractor_layer.trainable = True

    # setup model
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=IMAGE_SIZE+ (3,)),
    feature_extractor_layer,
    tf.keras.layers.Flatten(),
    # tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation='sigmoid')
    ])

model.build((None,)+IMAGE_SIZE+(3,))

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=['acc'])
    
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 2048)              21802784  
_________________________________________________________________
flatten (Flatten)            (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 2049      
Total params: 21,804,833
Trainable params: 21,770,401
Non-trainable params: 34,432
_________________________________________________________________


In [9]:
history_ft = model.fit(
            train_generator,
            epochs=EPOCHS_2,
            validation_data=validation_generator,
            steps_per_epoch=steps_per_epoch,
            validation_steps=val_steps,
            initial_epoch=EPOCHS_1,
            verbose=1,
            workers=4, 
            callbacks=[cp_callback, batch_stats, cvs_logger]
            )

Epoch 6/10
3125/3125 [==============================] - ETA: 0s - loss: 0.7838 - acc: 0.9375
Epoch 00006: val_acc improved from 0.82300 to 0.91190, saving model to Bi_Nilgai05202020-173418/cp-0006.ckpt
3125/3125 [==============================] - 2558s 819ms/step - loss: 0.7838 - acc: 0.9375 - val_loss: 0.7875 - val_acc: 0.9119
Epoch 7/10
3125/3125 [==============================] - ETA: 0s - loss: 0.6924 - acc: 0.9688
Epoch 00007: val_acc improved from 0.91190 to 0.92050, saving model to Bi_Nilgai05202020-173418/cp-0007.ckpt
3125/3125 [==============================] - 2698s 863ms/step - loss: 0.6924 - acc: 0.9688 - val_loss: 0.7517 - val_acc: 0.9205
Epoch 8/10
3125/3125 [==============================] - ETA: 0s - loss: 0.7240 - acc: 0.8750
Epoch 00008: val_acc improved from 0.92050 to 0.92800, saving model to Bi_Nilgai05202020-173418/cp-0008.ckpt
3125/3125 [==============================] - 2690s 861ms/step - loss: 0.7240 - acc: 0.8750 - val_loss: 0.7225 - val_acc: 0.9280
Epoch 9/10

In [10]:
EPOCHS_3 = EPOCHS_2 + 3
LEARNING_RATE = 0.00001

# load last checkpoint
# model = tf.keras.models.load_model(model_path, compile=True)

latest = tf.train.latest_checkpoint(checkpoint_dir)
model.load_weights(latest)
latest

'Bi_Nilgai05202020-173418\\cp-0009.ckpt'

In [11]:
# from previsous model training event
feature_extractor_layer.trainable = True

    # setup model
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=IMAGE_SIZE+ (3,)),
    feature_extractor_layer,
    tf.keras.layers.Flatten(),
    # tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation='sigmoid')
    ])

model.build((None,)+IMAGE_SIZE+(3,))

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=['acc'])
    
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 2048)              21802784  
_________________________________________________________________
flatten_1 (Flatten)          (None, 2048)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 2049      
Total params: 21,804,833
Trainable params: 21,770,401
Non-trainable params: 34,432
_________________________________________________________________


In [12]:
history_ft = model.fit(
            train_generator,
            epochs=EPOCHS_3,
            validation_data=validation_generator,
            steps_per_epoch=steps_per_epoch,
            validation_steps=val_steps,
            initial_epoch=EPOCHS_2,
            verbose=1,
            workers=4, 
            callbacks=[cp_callback, batch_stats, cvs_logger]
            )

Epoch 11/13
3125/3125 [==============================] - ETA: 0s - loss: 0.6644 - acc: 1.0000
Epoch 00011: val_acc improved from 0.93270 to 0.96060, saving model to Bi_Nilgai05202020-173418/cp-0011.ckpt
3125/3125 [==============================] - 2531s 810ms/step - loss: 0.6644 - acc: 1.0000 - val_loss: 0.6820 - val_acc: 0.9606
Epoch 12/13
3125/3125 [==============================] - ETA: 0s - loss: 0.6852 - acc: 1.0000
Epoch 00012: val_acc improved from 0.96060 to 0.96130, saving model to Bi_Nilgai05202020-173418/cp-0012.ckpt
3125/3125 [==============================] - 2542s 814ms/step - loss: 0.6852 - acc: 1.0000 - val_loss: 0.6793 - val_acc: 0.9613
Epoch 13/13
3125/3125 [==============================] - ETA: 0s - loss: 0.6795 - acc: 0.9688
Epoch 00013: val_acc improved from 0.96130 to 0.96430, saving model to Bi_Nilgai05202020-173418/cp-0013.ckpt
3125/3125 [==============================] - 2556s 818ms/step - loss: 0.6795 - acc: 0.9688 - val_loss: 0.6730 - val_acc: 0.9643


In [13]:
# finte-tuned model file path
EXPORT_MODEL_PATH = "F:/Nilgai_photo_database/Nilgai Classifier/tf/FINAL_saved_model/FINALE_BI_" + MODEL_NAME + "/"

model.save(EXPORT_MODEL_PATH, save_format='tf')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: F:/Nilgai_photo_database/Nilgai Classifier/tf/FINAL_saved_model/FINALE_BI_Bi_Nilgai/assets
INFO:tensorflow:Assets written to: F:/Nilgai_photo_database/Nilgai Classifier/tf/FINAL_saved_model/FINALE_BI_Bi_Nilgai/assets


# Review Model metrics

In [3]:
# call training and validation generators
train_generator, test_generator = binary_datagen(TRAIN_CSV, TEST_CSV, BATCH_SIZE, IMAGE_SIZE)

Found 1000 validated image filenames belonging to 2 classes.
Found 10000 validated image filenames belonging to 2 classes.


In [12]:
model = tf.keras.models.load_model(r'F:/Nilgai_photo_database/Nilgai Classifier/tf/FINAL_saved_model/FINALE_BI_Bi_Nilgai', compile=True)
test_steps = np.ceil(test_generator.samples/test_generator.batch_size)
train_steps = np.ceil(train_generator.samples/train_generator.batch_size)
# loss, acc = model.evaluate(test_generator, steps=test_steps, verbose=1, workers=4)

In [4]:
pred = bi_predict(TEST_CSV, TRAIN_CSV, r'F:/Nilgai_photo_database/Nilgai Classifier/tf/FINAL_saved_model/FINALE_BI_Bi_Nilgai')

Found 10000 validated image filenames.
Found 1000 validated image filenames belonging to 2 classes.
10000/10000 [==============================] - 178s 18ms/step
Compare Prediction and Test CSV results saved here: F:/Nilgai_photo_database/Nilgai Classifier/tf/results\20200521-092140_10K_test_weighted_nil_data.csv_pred_results.csv
